In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
#Initialize Spark Session
spark = SparkSession.builder.appName("TestApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 06:43:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
findspark.find()

'/opt/anaconda3/lib/python3.11/site-packages/pyspark'

In [9]:
sc=spark.sparkContext

In [11]:
rdd = sc.parallelize([1,2,3,4])
rdd.collect()

[1, 2, 3, 4]

In [12]:
resultRdd = rdd.map(lambda a:a*2)
resultRdd.collect()

[2, 4, 6, 8]

In [16]:
resultRdd = rdd.flatMap(lambda x:(x,x*2))
resultRdd.collect()

[1, 2, 2, 4, 3, 6, 4, 8]

In [19]:
resultRdd = rdd.filter(lambda x: x%2!=0)
resultRdd.collect()

[1, 3]

In [24]:
rdd1 = sc.parallelize([(1,2),(2,3),(3,4),(1,5)])
resultRdd = rdd1.reduceByKey(lambda x,y: x+y)
resultRdd.collect()

[(2, 3), (1, 7), (3, 4)]

In [25]:
resultRdd = rdd1.groupByKey()
resultRdd.collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7f73e592b0d0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f73e592b2d0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f73e5923210>)]

In [26]:
for x,y in resultRdd.collect():
    print(f"Key:{x}, Values:{list(y)}")

Key:2, Values:[3]
Key:1, Values:[2, 5]
Key:3, Values:[4]


In [27]:
word_list = ["A","B","C","D","A","A","C"]

In [29]:
wordRdd = sc.parallelize(word_list)
wordRdd.collect()

['A', 'B', 'C', 'D', 'A', 'A', 'C']

In [41]:
countRdd = wordRdd.map(lambda x: (x,x)).groupByKey()
# countRdd.collect()

In [42]:
for key, val in countRdd.collect():
    print(f"{key} occurs {len(list(val))} times")

A occurs 3 times
D occurs 1 times
B occurs 1 times
C occurs 2 times


In [43]:
countRdd = wordRdd.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
for key, val in countRdd.collect():
    print(f"{key} occurs {(val)} times")

A occurs 3 times
D occurs 1 times
B occurs 1 times
C occurs 2 times


In [44]:
purchaseRdd = sc.textFile("/home/labuser/Downloads/purchases.csv")
purchaseRdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [57]:
testDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Downloads/purchases.csv")
testDf.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/21 08:59:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Downloads/purchases.csv


In [69]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField("",StringType(),True),
    StructField("apples",IntegerType(),True),
    StructField("oranges",IntegerType(),True),
])

In [70]:
testDf = spark.read.schema(schema).option("header",True).csv("/home/labuser/Downloads/purchases.csv")
testDf.show()

+------+------+-------+
|      |apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [71]:
testDf.printSchema()

root
 |-- : string (nullable = true)
 |-- apples: integer (nullable = true)
 |-- oranges: integer (nullable = true)



In [83]:
moviesDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Downloads/IMDB-Movie-Data.csv")
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [80]:
moviesSchema = StructType([
    StructField("Rank",IntegerType(),False),
    StructField("Title",StringType(),True),
    StructField("Genre",StringType(),True),
    StructField("Description",StringType(),True),
])

In [81]:
moviesDf = spark.read.schema(moviesSchema).option("header",True).csv("/home/labuser/Downloads/IMDB-Movie-Data.csv")
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)



In [84]:
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [85]:
from pyspark.sql.functions import *
moviesDf=moviesDf.withColumn("Rev_new",col("Revenue (Millions)"))
moviesDf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rev_new|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0| 333.13|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0

In [86]:
import datetime
moviesDf = moviesDf.withColumn("updated_timestamp",lit(datetime.datetime.now()))
moviesDf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rev_new|   updated_timestamp|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0| 333.13|2023-09-21 10:53:...|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log.